# Skin cancer image classification

the dataset would be downloaded throught kaggle, to complete this step i had download my kaggle API and upload it to the google colab note book i am using. The next step was to create a folder named kaggle and set the security preference. after doing this step we can download the *ISIC 2018 skin legion dataset* and unzip it. the dataset contains other folders like it's metadata and is 5gb large.

In [1]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
!pip install kaggle

In [2]:
!cp kaggle.json ~/.kaggle/

In [4]:
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
! kaggle datasets download kmader/skin-cancer-mnist-ham10000

skin-cancer-mnist-ham10000.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
!unzip skin-cancer-mnist-ham10000.zip

Archive:  skin-cancer-mnist-ham10000.zip
replace HAM10000_images_part_1/ISIC_0024306.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

## Loading in libraries and path

In [28]:
import os
from glob import glob
import pandas as pd
from skimage.io import imread
import matplotlib.pyplot  as plt
from PIL import Image
from pathlib import Path
import imagesize
import numpy as np

base_dir = os.path.join('../')

In [21]:
import tensorflow as tf
import tensorflow_hub as hub
print('Tf-version:', tf.__version__)

print("GPU","Yes Available" if tf.test.is_gpu_available() else "not Available")

Tf-version: 2.10.0
GPU Yes Available


2022-11-16 15:26:13.890544: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-16 15:26:13.890660: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [ ]:
!nvidia-smi

In [18]:
imageid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x for x in glob(os.path.join(base_dir, '*', '*.jpg'))}

lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}

In [19]:
base_dir

''

In [27]:
os.path


<module 'posixpath' from '/Users/favechibueze/miniforge3/envs/mike_env/lib/python3.8/posixpath.py'>

In [29]:
tile_df = pd.read_csv(os.path.join(base_dir, 'HAM10000_metadata.csv'))
tile_df['cell_type'] = tile_df['dx'].map(lesion_type_dict.get) 
tile_df['path'] = tile_df['image_id'].map(imageid_path_dict.get)
tile_df['cell_type_idx'] = pd.Categorical(tile_df['cell_type']).codes
tile_df.sample(3)

FileNotFoundError: [Errno 2] No such file or directory: '../HAM10000_metadata.csv'

In [ ]:
tile_df['cell_type'].unique()

In [ ]:
# load in all of the images
tile_df['image'] = tile_df['path'].map(imread)

In [ ]:
# see the image size distribution
tile_df['image'].map(lambda x: x.shape).value_counts()

In [ ]:
tile_df[['cell_type_idx', 'cell_type']].sort_values('cell_type_idx').drop_duplicates()

In [ ]:
tile_df['image'] = tile_df['path'].map(lambda x: np.asarray(Image.open(x).resize((224,224))))

## converting the size of the image dataset

because the dataset is 450 in height and 600 in width and in RGB format, i would like to downscale the size to help in reducing training time on the GPU

In [ ]:
n_samples = 3
fig, m_axs = plt.subplots(7, n_samples, figsize = (4*n_samples, 3*7))
for n_axs, (type_name, type_rows) in zip(m_axs, 
                                         tile_df.sort_values(['cell_type']).groupby('cell_type')):
    n_axs[0].set_title(type_name)
    for c_ax, (_, c_row) in zip(n_axs, type_rows.sample(n_samples, random_state=42).iterrows()):
        c_ax.imshow(c_row['image'])
        c_ax.axis('off')
fig.savefig('category_samples.png', dpi=300)

In [ ]:
# See the image size distribution - should just return one row (all images are uniform)
tile_df['image'].map(lambda x: x.shape).value_counts()

In [ ]:
tile_df.head(3)

In [ ]:
tile_df.dtypes

In [ ]:
tile_df.describe()

# DATA CLEANING

In [ ]:
tile_df.isnull().sum()

In [ ]:
import missingno as msno
msno.matrix(tile_df)

In [ ]:
tile_df['age'].fillna(int(tile_df['age'].mean()),inplace=True)

In [ ]:
tile_df.isnull().sum()

# UNIVARIATE ANALYSIS

In [ ]:
plt.figure(figsize=(20,10))
plt.subplots_adjust(left=0.125, bottom=1, right=0.9, top=2, hspace=0.2)
plt.subplot(2,4,1)
plt.title("AGE",fontsize=15)
plt.ylabel("Count")
tile_df['age'].value_counts().plot.bar()

plt.subplot(2,4,2)
plt.title("GENDER",fontsize=15)
plt.ylabel("Count")
tile_df['sex'].value_counts().plot.bar()

plt.subplot(2,4,3)
plt.title("localization",fontsize=15)
plt.ylabel("Count")
plt.xticks(rotation=45)
tile_df['localization'].value_counts().plot.bar()

plt.subplot(2,4,4)
plt.title("CELL TYPE",fontsize=15)
plt.ylabel("Count")
tile_df['cell_type'].value_counts().plot.bar()

In [ ]:
import seaborn as sns

In [ ]:
a=sns.catplot(data=tile_df, x="cell_type", y="localization")
a.set_xticklabels(rotation=90)

In [ ]:
b=sns.catplot(data=tile_df, x="cell_type", y="localization", jitter=False)
b.set_xticklabels(rotation=90)

In [ ]:
sns.catplot(
    data=tile_df, x="cell_type", hue="localization", kind = 'count',
    palette="pastel", edgecolor=".6",height=5, aspect=10/5
).set_xticklabels(rotation=90)


In [ ]:
sns.catplot(
    data=tile_df, x="localization", hue="cell_type", kind = 'count',
    palette="pastel", edgecolor=".6",height=5, aspect=10/5
).set_xticklabels(rotation=90)

In [ ]:

sns.catplot(data=tile_df, x="localization", kind="count", palette="ch:.25", height=5, aspect=10/5).set_xticklabels(rotation=90)

In [ ]:
plt.figure(figsize=(15,10))
plt.subplot(1,2,1)
tile_df['dx'].value_counts().plot.pie(autopct="%1.1f%%")
plt.subplot(1,2,2)
tile_df['dx_type'].value_counts().plot.pie(autopct="%1.1f%%")
plt.show()

In [ ]:
plt.figure(figsize=(25,10))
plt.title('LOCALIZATION VS GENDER',fontsize = 15)
sns.countplot(y='localization', hue='sex',data=tile_df)

In [ ]:
plt.figure(figsize=(25,10))
plt.title('LOCALIZATION VS CELL TYPE',fontsize = 15)
sns.countplot(x='localization', hue='cell_type',data=tile_df)

In [ ]:
plt.figure(figsize=(25,10))
plt.title('LOCALIZATION VS CELL TYPE',fontsize = 15)
sns.countplot(x='cell_type', hue='localization',data=tile_df)

In [ ]:
plt.figure(figsize=(25,10))
plt.subplot(131)
plt.title('AGE VS CELL TYPE',fontsize = 15)
sns.countplot(y='age', hue='cell_type',data=tile_df)
plt.subplot(132)
plt.title('GENDER VS CELL TYPE',fontsize = 15)
sns.countplot(y='sex', hue='cell_type',data=tile_df)

# ANN 

In [ ]:
import pandas as pd
from sklearn.datasets import load_iris

In [ ]:
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import seaborn as sns
import numpy as np
import pandas as pd
import os
from tensorflow.keras.utils import to_categorical
from glob import glob

In [ ]:
trial_data = tile_df.sample(3000)

In [ ]:
features = trial_data.drop(['cell_type_idx'], axis = 1)
label = trial_data['cell_type_idx']

In [ ]:
features.shape,label.shape

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(features, label,
                                                    test_size=0.2, shuffle = True, random_state = 42)



In [ ]:
print("X_train shape: {}".format(x_train.shape))
print("X_test shape: {}".format(x_test.shape))
print("y_train shape: {}".format(y_train.shape))
print("y_test shape: {}".format(y_test.shape))

In [ ]:
tf.unique(x_train.cell_type.values)

In [ ]:
x_train = np.asarray(x_train['image'].tolist())
x_test = np.asarray(x_test['image'].tolist())

x_train_mean = np.mean(x_train)
x_train_std = np.std(x_train)

x_test_mean = np.mean(x_test)
x_test_std = np.std(x_test)

x_train = (x_train - x_train_mean)/x_train_std
x_test = (x_test - x_test_mean)/x_test_std

In [ ]:
x_train.shape

In [ ]:
y_train = to_categorical(y_train, num_classes = 7)
y_test = to_categorical(y_test, num_classes = 7)

y_test.shape, y_train.shape

In [ ]:
y_test

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.25, random_state = 42)

In [ ]:
x_train = x_train.reshape(x_train.shape[0],x_train.shape[1] * x_train.shape[2] * x_train.shape[3])
x_test = x_test.reshape(x_test.shape[0],x_test.shape[1] * x_test.shape[2] * x_test.shape[3])
x_val = x_val.reshape(x_val.shape[0],x_val.shape[1] * x_val.shape[2] * x_val.shape[3])
print(x_train.shape)
print(x_test.shape)


In [ ]:
x_val.shape

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(x_val.shape)
print(y_train.shape)
print(y_test.shape)
print(y_val.shape)

# MODEL 1

In [ ]:
tf.random.set_seed(28)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(units= 16, kernel_initializer = 'uniform', activation = 'relu'),
    tf.keras.layers.Dense(units= 32, kernel_initializer = 'uniform', activation = 'relu'),
    tf.keras.layers.Dense(units= 64, kernel_initializer = 'uniform', activation = 'relu'),
    tf.keras.layers.Dense(units= 128, kernel_initializer = 'uniform', activation = 'relu'),
    #tf.keras.layers.Dense(units= 512, kernel_initializer = 'uniform', activation = 'relu'),
    tf.keras.layers.Dense(units = 7, kernel_initializer = 'uniform', activation = 'softmax')])

optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001,
                                    beta_1 = 0.9,
                                    beta_2 = 0.999,
                                    epsilon = 1e-8)

# compile the keras model
model.compile(optimizer = optimizer, 
              loss = tf.keras.losses.CategoricalCrossentropy(),
              metrics = ['accuracy'])

# fit the keras model on the dataset
epochs = 100
history = model.fit(x_train,
                    y_train, 
                    batch_size = 100, 
                    epochs = epochs,
                    validation_data = (x_val,y_val)) #, # verbose = 0)


In [ ]:
accuracy = model.evaluate(x_test, y_test, verbose=0)[1]
print("Test: accuracy = ",accuracy*100,"%")

In [ ]:
# Plot the learning rate versus the loss
lrs = 1e-4 * (10 ** (np.arange(epochs)/20))
plt.figure(figsize=(10, 7))
plt.semilogx(lrs, history.history["loss"]) # we want the x-axis (learning rate) to be log scale
plt.xlabel("Learning Rate")
plt.ylabel("Loss")
plt.title("Learning rate vs. loss");

In [ ]:
import pandas as pd
pd.DataFrame(history.history).plot(title='Rates', figsize=(15, 7))

In [ ]:
tf.random.set_seed(22)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(units= 8, kernel_initializer = 'uniform', activation = 'relu'),
    tf.keras.layers.Dense(units= 16, kernel_initializer = 'uniform', activation = 'relu'),
    tf.keras.layers.Dense(units= 32, kernel_initializer = 'uniform', activation = 'relu'),
    tf.keras.layers.Dense(units= 64, kernel_initializer = 'uniform', activation = 'relu'),
    #tf.keras.layers.Dense(units= 512, kernel_initializer = 'uniform', activation = 'relu'),
    tf.keras.layers.Dense(units = 7, kernel_initializer = 'uniform', activation = 'softmax')])

optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001,
                                    beta_1 = 0.9,
                                    beta_2 = 0.999,
                                    epsilon = 1e-8)

# compile the keras model
model.compile(optimizer = optimizer, 
              loss = tf.keras.losses.CategoricalCrossentropy(),
              metrics = ['accuracy'])

# fit the keras model on the dataset
history = model.fit(x_train,
                    y_train, 
                    batch_size = 100, 
                    epochs = 90,
                    validation_data = (x_val,y_val)) #, # verbose = 0)

# MODEL 2

In [ ]:
model1 = tf.keras.Sequential([
    tf.keras.layers.Dense(units= 16, kernel_initializer = 'uniform', activation = 'relu'),
    tf.keras.layers.Dense(units= 32, kernel_initializer = 'uniform', activation = 'relu'),
    tf.keras.layers.Dense(units= 32, kernel_initializer = 'uniform', activation = 'relu'),
    tf.keras.layers.Dense(units= 64, kernel_initializer = 'uniform', activation = 'relu'),
    #tf.keras.layers.Dense(units= 512, kernel_initializer = 'uniform', activation = 'relu'),
    tf.keras.layers.Dense(units = 7, kernel_initializer = 'uniform', activation = 'softmax')])

optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001,
                                    beta_1 = 0.9,
                                    beta_2 = 0.999,
                                    epsilon = 1e-8)

# compile the keras model
model1.compile(optimizer = optimizer, 
              loss = tf.keras.losses.CategoricalCrossentropy(),
              metrics = ['accuracy'])

# fit the keras model on the dataset
epochs = 100
history1 = model1.fit(x_train,
                    y_train, 
                    batch_size = 100, 
                    epochs = epochs,
                    validation_data = (x_val,y_val)) #, # verbose = 0)

In [ ]:
accuracy1 = model.evaluate(x_test, y_test, verbose=0)[1]
print("Test: accuracy = ",accuracy1*100,"%")

In [ ]:
# Plot the learning rate versus the loss
lrs1 = 1e-4 * (10 ** (np.arange(epochs)/20))
plt.figure(figsize=(10, 7))
plt.semilogx(lrs1, history1.history["loss"]) # we want the x-axis (learning rate) to be log scale
plt.xlabel("Learning Rate")
plt.ylabel("Loss")
plt.title("Learning rate vs. loss");

In [ ]:
import pandas as pd
pd.DataFrame(history1.history).plot(title='Rates', figsize=(15, 7))

In [ ]:
tf.random.set_seed(42)

model1 = tf.keras.Sequential([
    tf.keras.layers.Dense(units= 32, kernel_initializer = 'uniform', activation = 'relu'),
    tf.keras.layers.Dense(units= 64, kernel_initializer = 'uniform', activation = 'relu'),
    tf.keras.layers.Dense(units= 128, kernel_initializer = 'uniform', activation = 'relu'),
    tf.keras.layers.Dense(units= 256, kernel_initializer = 'uniform', activation = 'relu'),
    tf.keras.layers.Dense(units = 7, kernel_initializer = 'uniform', activation = 'softmax')])

optimizer = tf.keras.optimizers.Adam(learning_rate = 0.00075,
                                    beta_1 = 0.9,
                                    beta_2 = 0.999,
                                    epsilon = 1e-8)

# compile the keras model
model1.compile(optimizer = optimizer, 
              loss = tf.keras.losses.CategoricalCrossentropy(),
              metrics = ['accuracy'])

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-3 * 10 ** (epoch/20))

# fit the keras model on the dataset
history1 = model1.fit(x_train, 
                      y_train, 
                      batch_size = 10, 
                      epochs = 65, 
                      verbose = 0,
                      callbacks = [lr_scheduler])

accuracy = model1.evaluate(x_val, y_val, verbose=0)[1]
print("Test: accuracy = ",accuracy*100,"%")



In [ ]:
lrs = 1e-3 * (10**(tf.range(65)/20))
plt.semilogx(lrs,history1.history['loss'])
plt.xlabel('Learning Rate')
plt.ylabel('Loss')
plt.title('Finding the ideal learning rate')

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
import itertools
from sklearn.metrics import confusion_matrix

def make_confusion_matrix(y_true,y_pred, classes = None, figsize = (10,10), text_size=15):
  cm = confusion_matrix(y_true,tf.round(y_pred))
  cm_norm = cm.astype('float') / cm.sum(axis=1)[:,np.newaxis]
  n_classes = cm.shape[0]

  # Making it pretty
  fig,ax = plt.subplots(figsize = figsize)
  cax = ax.matshow(cm,cmap=plt.cm.Blues)
  fig.colorbar(cax)

# Set labels to classes
  if classes:
    labels = classes
  else:
    labels = np.arange(cm.shape[0])

  # Label the axes
  ax.set(title = 'Confusion Matrix',
        xlabel = 'Predicted Label',
        ylabel= 'True Label',
        xticks = np.arange(n_classes),
        yticks = np.arange(n_classes),
        xticklabels = labels,
        yticklabels = labels)

  # Set x-axis labels to bottom
  ax.xaxis.set_label_position('bottom')
  ax.xaxis.tick_bottom()

  # Adjust label size
  ax.yaxis.label.set_size(text_size)
  ax.xaxis.label.set_size(text_size)
  ax.title.set_size(text_size)

  # Set threshold for different colors
  threshold = (cm.max() + cm.min())/2

  # Plot the text on each
  for i,j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j,i, f"{cm[i,j]} ({cm_norm[i,j]*100:.1f}%)",
            horizontalalignment = 'center',
            color = 'white' if cm[i,j] > threshold else 'black',
            size = text_size)

In [ ]:
y_probs = model.predict(y_test) 
y_preds = y_probs .argmax(axis=1)

In [ ]:
class_names = ['Benign keratosis-like lesions ', 'Melanocytic nevi','Dermatofibroma', 
               'Melanoma', 'Vascular lesions','Basal cell carcinoma', 'Actinic keratoses']
make_confusion_matrix(y_true=y_test, 
                      y_pred = y_preds, 
                      classes = class_names,
                      figsize = (13,13),
                      text_size = 10)

# TRANSFER LEARNING

#CNN

In [ ]:
from tensorflow.keras.layers import Flatten,Dense,Dropout,BatchNormalization,Conv2D, MaxPool2D
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
input_shape = (224, 224, 3)
num_classes = 7

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',padding = 'Same'))
model.add(Conv2D(32,kernel_size=(3, 3), activation='relu',padding = 'Same'))
model.add(MaxPool2D(pool_size = (2, 2)))
model.add(Dropout(0.16))

model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',padding = 'Same'))
model.add(Conv2D(32,kernel_size=(3, 3), activation='relu',padding = 'Same',))
model.add(MaxPool2D(pool_size = (2, 2)))
model.add(Dropout(0.20))

model.add(Conv2D(64, (3, 3), activation='relu',padding = 'same'))
model.add(Conv2D(64, (3, 3), activation='relu',padding = 'Same'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

In [ ]:
# Define the optimizer
optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

In [ ]:
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=4, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)